In [11]:
!pip install langchain langgraph langchain-openai openai faiss-cpu requests gradio --quiet

In [12]:
import os
import getpass
import requests
import gradio as gr

# Securely input API keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("\U0001F511 Enter your OpenAI API key: ")
os.environ["WEATHER_API_KEY"] = getpass.getpass("\U0001F327️ Enter your OpenWeather API key: ")





🔑 Enter your OpenAI API key: ··········
🌧️ Enter your OpenWeather API key: ··········


In [13]:
def get_weather(city):
    api_key = os.getenv("WEATHER_API_KEY")
    if not api_key:
        raise ValueError("Weather API key not set.")

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code != 200 or "main" not in data:
        raise ValueError(data.get("message", "Unknown weather error."))

    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    return temp, desc




In [14]:
# for aqi level

def get_aqi(city):
    api_key = os.getenv("WEATHER_API_KEY")
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
    geo_resp = requests.get(geo_url).json()
    if not geo_resp:
        raise ValueError("Invalid city for AQI.")

    lat, lon = geo_resp[0]["lat"], geo_resp[0]["lon"]
    aqi_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
    aqi_resp = requests.get(aqi_url).json()

    aqi_index = aqi_resp["list"][0]["main"]["aqi"]
    levels = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}
    return f"AQI in {city}: {levels.get(aqi_index, 'Unknown')}"


In [15]:
# carbon estimation
def estimate_carbon(mode, distance_km):
    rates = {"car": 0.21, "air-plane": 0.15, "bus": 0.1, "train": 0.05, "bike": 0.0, "walk": 0.0}
    emission = rates.get(mode, 0.21) * distance_km
    return f"Estimated CO₂: {emission:.2f} kg"




In [16]:
# smart suggestion

def suggest_transport(weather_desc, distance_km):
    if distance_km < 2:
        return "🚶 Suggestion: Walk"
    elif distance_km < 5 and "rain" not in weather_desc.lower():
        return "🚴 Suggestion: Bike"
    elif "rain" in weather_desc.lower():
        return "🚌 Suggestion: Take the bus (bad weather)"
    else:
        return "🚇 Suggestion: Use train or shared transport"

In [17]:
from langchain_openai import OpenAI
from langchain.agents import Tool, initialize_agent

llm = OpenAI(temperature=0.5)

tools = [
    Tool(
        name="WeatherTool",
        func=lambda city: f"Weather in {city}: {get_weather(city)[1]}",
        description="Get weather description for a given city."
    ),
    Tool(
        name="CarbonTool",
        func=lambda tool_input: estimate_carbon(tool_input['mode'], tool_input['distance']),
        description="Estimate CO₂ for a trip. Input: {'mode': str, 'distance': float}"
    ),
    Tool(
        name="AQITool",
        func=get_aqi,
        description="Fetch Air Quality Index for a given city."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [18]:
goals_done = []

# Gradio Interface

def eco_agent_interface(city, mode, distance, goal_done):
    try:
        temp, weather_desc = get_weather(city)
        carbon = estimate_carbon(mode, float(distance))
        suggestion = suggest_transport(weather_desc, float(distance))
        aqi_info = get_aqi(city)

        if goal_done:
            goals_done.append((city, mode))
            goal_msg = "✅ You completed today's climate-friendly action!"
        else:
            goal_msg = "🔁 Try completing your daily eco action tomorrow!"

        return f"""\n🌡️ Weather in {city}: {temp}°C, {weather_desc}\n{aqi_info}\n\n{carbon}\n{suggestion}\n\n{goal_msg}"""

    except Exception as e:
        return f"❌ Error: {str(e)}"



In [19]:
#launch web app

gr.Interface(
    fn=eco_agent_interface,
    inputs=[
        gr.Textbox(label="City Name"),
        gr.Dropdown(["car", "air-plane", "bus", "train", "bike", "walk"], label="Transport Mode"),
        gr.Number(label="Distance in km"),
        gr.Checkbox(label="Did you complete your eco-friendly action today?")
    ],
    outputs="text",
    title="🌿 EcoMate Pro – Agentic AI for Climate Action",
    description="Get real-time weather, AQI, carbon impact & personalized transport suggestions"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb421fb57ba1c628e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
